<a href="https://colab.research.google.com/github/StuartGJohnson/ArduinoTensorFlowLiteTutorials/blob/master/ingest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


OK, lets see if we can get some of these pdf ingestion tools to work on our docs. And figure out how to see what they are seeing and not seeing!

In [2]:
import os

In [3]:
os.listdir('/content/drive/MyDrive/FDA')

['Premarket-Software-Functions-Guidance.pdf',
 'General-Principles-of-Software-Validation---Final-Guidance-for-Industry-and-FDA-Staff.pdf',
 'Cybersecurity-for-Networked-Medical-Devices-Containing-Off-the-Shelf-(OTS)-Software---Guidance-for-Industry.pdf',
 'Deciding-When-to-Submit-a-510(k)-for-a-Software-Change-to-an-Existing-Device---Guidance-for-Industry-and-Food-and-Drug-Administration-Staff.pdf',
 'Off-The-Shelf-Guidance_0.pdf',
 'Guidance-Clinical-Decision-Software.pdf',
 'Guidance-DSF-MMA.pdf',
 'Guidance-Computer-Software-Assurance.pdf',
 'Guidance-Devices-3060_0.pdf',
 'Software-as-a-Medical-Device-(SAMD)--Clinical-Evaluation---Guidance-for-Industry-and-Food-and-Drug-Administration-Staff.pdf',
 '.tmp.driveupload']

In [5]:
!pip install pypdf
!pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 811.8/811.8 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 46.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.4/239.4 kB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.7/55.7 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 6.2 MB/s eta 0:00:00


In [6]:
from langchain.document_loaders import PyPDFLoader

In [10]:
loader  = PyPDFLoader("/content/drive/MyDrive/FDA/Off-The-Shelf-Guidance_0.pdf")

In [26]:
docs = loader.load()

In [27]:
len(docs)

20

split up the text.

In [28]:
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs_split = text_splitter.split_documents(docs)

In [19]:
!pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.8/132.8 kB 3.3 MB/s eta 0:00:00


In [20]:
# Embeddings
from langchain.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [21]:
!pip install faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 30.1 MB/s eta 0:00:00


In [32]:
from langchain.vectorstores import FAISS

db = FAISS.from_documents(docs_split, embeddings)

query = "LAN Architecture"
docs_sim = db.similarity_search(query)

In [36]:
docs_sim[0]

Document(page_content='Contains  Nonbinding  Recommendations\n18E. Data  Integrity  – One of the most  important  issues  for any medical  device  operating  in a \nnetwork is data integrity. The manufacturer should ensure that the network system software  \nand hardware  incorporate  error  checking,  handling,  and correction  measures as determined by a \ncybersecurity risk assessment .\n1. Transmission  of data packets  and files should  include  error  detection  \nand protection  measures , such as digi tal signatures  or cryptographic \nintegrity checks . \n2. Transaction  rollback  after non-committed  changes  or network  failure,  \nsupports data integrity in medical device LANs.\n3. Critical  data and files  may be  stored  in duplicate  at separate locations.\nF. Network  Management  and Security  – User  authorization  and authentication  \nshould precede accesses to sensitive patient information.\nThe above  five items  are not independent.  Decisions  made  in one item a